In [1]:
from math import sqrt, ceil
import geopandas as gpd
from geopandas import sjoin
from os.path import join
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import libpysal
import warnings
from folders import lyr_fldr, model_fldr, model_file, hexbins_file, final_zoning

ImportError: DLL load failed: The specified procedure could not be found.

In [ ]:
hexbins_file = join(lyr_fldr, 'hexbins_small.gpkg')

# Zone size parameters

In [ ]:
max_zone_size = 10000 # No zone can have more than 50,000 people
min_zone_size = 500 # No zone can have less than 500 people

# Input data

In [ ]:
%%time
hexbins = gpd.read_file(hexbins_file, layer='hexbins', driver='gpkg')

# Initializes centroids zoned by district
It could've (should've) been by distritict, but let's try to face the computational challenge!

In [ ]:
%%time
hexbins['zone_id'] = -1
list_districts = list(hexbins.district.unique())
data_store = []
master_zone_id = 1
for i, district in enumerate(list_districts):
    df = hexbins[hexbins.district==district].copy()
    df.loc[:, 'zone_id'] = master_zone_id + i
    data_store.append(df[['hex_id', 'x', 'y', 'population', 'province', 'district', 'zone_id']])

# And progressively break them

In [ ]:
%%time
for cnt, df in enumerate(data_store):
    t = df.groupby(['zone_id']).sum()
    t = t.loc[t.population>max_zone_size]
    zone_sizes = t['population'].to_dict()
    zones_to_break = len(zone_sizes)
    counter = 0
    threshold = 500
    thrsh = 500
    if cnt % 25 == 0:
        print(f'Done {cnt}/{len(data_store)} districts')
    while zones_to_break > 0:
        counter += 1
        zone_to_analyze = min(zone_sizes)
        zone_pop = zone_sizes.pop(zone_to_analyze)
        zones_to_break -= 1
        if zone_pop < max_zone_size:
            continue
        fltr = df.zone_id==zone_to_analyze
        segments = max(2, ceil(sqrt(zone_pop/max_zone_size)))
        prov_pop = df.loc[fltr, :]
        segments = min( prov_pop.shape[0], segments)
        if prov_pop.shape[0] < 2:
            continue
        
        kmeans = KMeans(n_clusters=segments, random_state=0)
        centr_results = kmeans.fit_predict(X=prov_pop[['x', 'y']].values, sample_weight=prov_pop.population.values)
        df.loc[fltr, 'zone_id'] = centr_results[:] + master_zone_id

        t = df.groupby(['zone_id']).sum()
        ready= t.loc[t.population<=max_zone_size].shape[0]
        avg = int(np.nansum(t.loc[t.population<=max_zone_size, 'population'])/max(1, ready))
        t = t.loc[t.population>max_zone_size]
        zone_sizes = t['population'].to_dict()
        zones_to_break = len(zone_sizes)
        master_zone_id += segments + 1
        if counter % 50 == 0:
            print(f'Queue for analysis: {zones_to_break} (Done: {ready} ({avg} people/zone))')

In [ ]:
%%time
df = pd.concat(data_store)[['hex_id', 'zone_id']]
df = pd.merge(hexbins[['hex_id', 'x', 'y', 'population', 'province', 'district', 'geometry']], df, on='hex_id')
df = gpd.GeoDataFrame(df[['hex_id', 'x', 'y', 'population', 'province', 'district', 'zone_id']], geometry=df['geometry'])

In [ ]:
%%time
zoning=df.dissolve(by='zone_id')[['province', 'district', 'geometry']]
pop_total = df[['zone_id', 'population']].groupby(['zone_id']).sum()['population']
zoning = zoning.join(pop_total)

# Break non-contiguous zones

In [ ]:
%%time
while zoning[zoning.geometry.type=='MultiPolygon'].shape[0] > 0:
    for zid, record in zoning[zoning.geometry.type=='MultiPolygon'].iterrows():
        zone_df = df[df.zone_id==zid]
        with warnings.catch_warnings():
            adj_mtx = libpysal.weights.Queen.from_dataframe(zone_df)
        island_pop = {isl:zone_df[adj_mtx.component_labels==isl].population.sum() for isl in islands}
        islands =  np.unique(adj_mtx.component_labels)
        max_island = max(island_pop.values())
        remove_islands = [k for k, v in island_pop.items() if v < max_island]
        for rmv in remove_islands:
            island_hexbins = zone_df[adj_mtx.component_labels == rmv].hex_id
            if zone_df[df.hex_id.isin(island_hexbins)].population.sum() > min_zone_size:
                df.loc[df.hex_id.isin(island_hexbins),'zone_id'] = master_zone_id
                master_zone_id += 1
                continue

            closeby = []
            for island_geo in zone_df[adj_mtx.component_labels == rmv].geometry:
                closeby.extend([x for x in df.sindex.nearest(island_geo.bounds, 6)])
            closeby = list(set(closeby))
            if not closeby:
                continue
            adjacent = df.loc[df.index.isin(closeby),:]
            available = [x for x in adjacent.zone_id.unique() if x != zid]

            same_area = [av for av in available if adjacent.loc[adjacent.zone_id==av, 'district'].values[0] == record.district]
            if same_area:
                df.loc[df.hex_id.isin(island_hexbins),'zone_id'] = same_area[0]
            else:
                counts = adjacent.groupby(['zone_id']).count()
                counts = list(counts[counts.hex_id==counts.hex_id.max()].index)
                counts = [x for x in counts if x != zid][0]
                df.loc[df.hex_id.isin(island_hexbins), 'district'] = adjacent.loc[adjacent.zone_id==counts, 'district'].values[0]
                df.loc[df.hex_id.isin(island_hexbins), 'province'] = adjacent.loc[adjacent.zone_id==counts, 'province'].values[0]
                df.loc[df.hex_id.isin(island_hexbins),'zone_id'] = counts

    zoning=df.dissolve(by='zone_id')[['province', 'district', 'geometry']]
    pop_total = df[['zone_id', 'population']].groupby(['zone_id']).sum()['population']
    zoning = zoning.join(pop_total)

In [ ]:
zoning=df.dissolve(by='zone_id')[['province', 'district', 'geometry']]
pop_total = df[['zone_id', 'population']].groupby(['zone_id']).sum()['population']
zoning = zoning.join(pop_total)

# Saves results

In [ ]:
%%time
zoning['final_zone_id'] = np.arange(zoning.shape[0]) + 1
zoning = gpd.GeoDataFrame(zoning[['final_zone_id', 'province', 'district', 'population']], geometry=zoning['geometry'])
zoning.to_file(final_zoning, driver='GPKG')

In [ ]:
# %%time
df.to_file(join(lyr_fldr, 'hexbins_with_zone.gpkg'), driver='GPKG')